In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
from tensorflow.python.keras import models
from tensorflow.keras import layers
from tensorflow.python.keras import losses
from tensorflow.keras import optimizers
from tensorflow.python.keras import activations
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import initializers
from tensorflow.python.keras import backend
from tensorflow.python.keras import metrics
from tensorflow.python import keras
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.model_selection import train_test_split

#%%

train_x_rc = np.load('../input/ventilator-traindata/train_x_rc.npy')
train_x_other = np.load('../input/ventilator-traindata/train_x_other.npy')
targets = np.load('../input/ventilator-traindata/target.npy')
print(train_x_other.shape)
print(train_x_rc.shape)
print(targets.shape)

#%%

rc_num = 15
ox = train_x_ther.shape[-1]
train_x = np.concatenate([train_x_rc, train_x_other], axis=-1)
x_train, x_test, y_train, y_test = train_test_split(train_x, targets, test_size=0.5, random_state=43)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.75, random_state=43)
x_train_rc = x_train[:, :, :rc_num][:, 0, :]
x_train_ox = x_train[:, :, -ox:]
x_test_rc = x_test[:, :, :rc_num][:, 0, :]
x_test_ox = x_test[:, :, -ox:]

In [ ]:
#%%

class ExpandTileLayer(layers.Layer):
    def __init__(self):
        super(ExpandTileLayer, self).__init__()
    def call(self, inputs, *args, **kwargs):
        return backend.tile(backend.expand_dims(inputs, axis=-2), (1, 80, 1))

#%%

In [ ]:

def build_model(hp):
    rc_input = keras.Input(shape=(train_x_rc.shape[-1], ),
                           dtype='float32',
                           name='rc_input_layer')
    other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
                                dtype='float32',
                                name='other_x_input')
    rc_embedding_layer = layers.Dense(units=hp.Choice('units_emb', [8, 16, 32]),
                                      use_bias=False, name='rc_embed_layer')
    rc_embedding = rc_embedding_layer(rc_input)

    # 维度拓展
    rc_embedding = ExpandTileLayer()(rc_embedding)

    x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
    ox = layers.Bidirectional(layers.LSTM(units=hp.Choice('units_0', [64, 128, 256]),
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(x_input)
    for i in range(hp.Int('num_layers', min_value=1, max_value=4, step=1)):
        ox = layers.Bidirectional(layers.LSTM(units=hp.Choice('units_'+str(i), [64, 128, 256]),
                                              return_sequences=True,
                                              kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                                  merge_mode='concat')(ox)
    ox = layers.Dense(units=hp.Choice('units_d1', [64, 128, 256]),
                      kernel_initializer=initializers.initializers_v2.GlorotUniform())(ox)
    if hp.Choice('d2_activation', ['prelu', 'elu']) == 'prelu':
        lstm_output = layers.PReLU()(ox)
    else:
        lstm_output = layers.ELU()(ox)

    conv1d_1 = layers.Conv1D(filters=hp.Choice('conv1d_1_filter', [64, 128, 256]),
                             kernel_size=hp.Choice('cov1d_1_size', [5, 11]),
                             padding='same', use_bias=False)(x_input)
    conv1d_1 = layers.BatchNormalization()(conv1d_1)
    conv1d_1 = layers.ReLU()(conv1d_1)
    conv1d_2 = layers.Conv1D(filters=hp.Choice('conv1d_2_filter', [64, 128, 256]), padding='same', use_bias=False,
                             kernel_size=hp.Choice('conv1d_2_size', [5, 7]))(conv1d_1)
    conv1d_2 = layers.BatchNormalization()(conv1d_2)
    conv1d_output = layers.ReLU()(conv1d_2)

    output = layers.Concatenate(axis=-1)([lstm_output, conv1d_output])
    output = layers.Dense(units=hp.Choice('final_units', [128, 256]),
                          activation=activations.relu,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
    output = layers.Dense(units=1,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

    my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])
    my_model.compile(loss=losses.MeanAbsoluteError(),
                     optimizer=optimizers.Adam(),
                     metrics=[metrics.MeanAbsoluteError()])
    return my_model


In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    tuner = kt.BayesianOptimization(
        build_model,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        project_name='ventilator'
    )

#%%

print(tuner.search_space_summary())

#%%



In [ ]:
tuner.search([x_train_rc, x_train_ox], y_train,
             validation_data=([x_test_rc, x_test_ox], y_test),
             epochs=80,
             batch_size=1024)
#%%

hp = tuner.get_best_hyperparameters(5)
pd.to_pickle(hp[0], './tuner_best_model1')
pd.to_pickle(hp[1], './tuner_best_model2')
pd.to_pickle(hp[2], './tuner_best_model3')
pd.to_pickle(hp[3], './tuner_best_model4')
pd.to_pickle(hp[4], './tuner_best_model5')